In [1]:
import pandas as pd
import json
pd.set_option("display.max_columns", 1000)
from flatten_json import flatten

In [15]:
with open("legislators-current.json", "r") as f:
    current_data = json.load(f)

with open("legislators-historical.json", "r") as f:
    historical_data = json.load(f)

In [16]:
current_df = pd.DataFrame([flatten(d) for d in current_data])

In [21]:
pd.json_normalize(current_data)

,terms,id.bioguide,id.thomas,id.lis,id.govtrack,id.opensecrets,id.votesmart,id.fec,id.cspan,id.wikipedia,id.house_history,id.ballotpedia,id.maplight,id.icpsr,id.wikidata,id.google_entity_id,name.first,name.last,name.official_full,bio.birthday,bio.gender,name.middle,name.nickname,name.suffix,leadership_roles,other_names,family
0,"[{'type': 'rep', 'start': '1993-01-05', 'end':...",B000944,00136,S307,400050,N00003535,27018.0,"[H2OH13033, S6OH00163]",5051.0,Sherrod Brown,9996.0,Sherrod Brown,168.0,29389.0,Q381880,kg:/m/034s80,Sherrod,Brown,Sherrod Brown,1952-11-09,M,NaN,NaN,NaN,NaN,NaN,NaN
1,"[{'type': 'rep', 'start': '1993-01-05', 'end':...",C000127,00172,S275,300018,N00007836,27122.0,"[S8WA00194, H2WA01054]",26137.0,Maria Cantwell,10608.0,Maria Cantwell,544.0,39310.0,Q22250,kg:/m/01x68t,Maria,Cantwell,Maria Cantwell,1958-10-13,F,NaN,NaN,NaN,NaN,NaN,NaN
2,"[{'type': 'rep', 'start': '1987-01-06', 'end':...",C000141,00174,S308,400064,N00001955,26888.0,"[H6MD03177, S6MD03177]",4004.0,Ben Cardin,10629.0,Ben Cardin,182.0,15408.0,Q723295,kg:/m/025k3k,Benjamin,Cardin,Benjamin L. Cardin,1943-10-05,M,L.,NaN,NaN,NaN,NaN,NaN
3,"[{'type': 'rep', 'start': '1983-01-03', 'end':...",C000174,00179,S277,300019,N00012508,22421.0,[S8DE00079],663.0,Tom Carper,10671.0,Tom Carper,545.0,15015.0,Q457432,kg:/m/01xw7t,Thomas,Carper,Thomas R. Carper,1947-01-23,M,Richard,NaN,NaN,NaN,NaN,NaN
4,"[{'type': 'sen', 'start': '2007-01-04', 'end':...",C001070,01828,S309,412246,N00027503,2541.0,[S6PA00217],47036.0,Bob Casey Jr.,NaN,"Bob Casey, Jr.",727.0,40703.0,Q887841,kg:/m/047ymw,Robert,Casey,"Robert P. Casey, Jr.",1960-04-13,M,P.,Bob,Jr.,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,"[{'type': 'rep', 'start': '2022-01-18', 'end':...",C001127,NaN,NaN,456865,NaN,NaN,[H8FL20032],NaN,Sheila Cherfilus-McCormick,NaN,NaN,NaN,NaN,Q109560948,NaN,Sheila,Cherfilus-McCormick,Sheila Cherfilus-McCormick,1979-01-25,F,NaN,NaN,NaN,NaN,NaN,NaN
533,"[{'type': 'rep', 'start': '2022-06-07', 'end':...",C001128,NaN,NaN,456866,NaN,NaN,[H2CA22231],NaN,Connie Conway,NaN,Connie Conway,NaN,NaN,Q5161840,NaN,Connie,Conway,Connie Conway,1950-09-25,F,NaN,NaN,NaN,NaN,NaN,NaN
534,"[{'type': 'rep', 'start': '2022-06-21', 'end':...",F000473,NaN,NaN,456867,NaN,NaN,[H2TX34113],NaN,Mayra Flores,NaN,NaN,NaN,NaN,Q112577890,NaN,Mayra,Flores,Mayra Flores,1986-01-01,F,NaN,NaN,NaN,NaN,NaN,NaN
535,"[{'type': 'rep', 'start': '2022-07-12', 'end':...",F000474,NaN,NaN,456868,NaN,NaN,[H2NE01118],NaN,Mike Flood (politician),NaN,Mike Flood,NaN,NaN,Q6846855,NaN,Mike,Flood,NaN,1975-02-23,M,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
current_df = pd.json_normalize(current_data, 
record_path=["terms"],
meta=[["id", "bioguide"],["bio", "birthday"], ["bio","gender"], ["name", "first"], ["name","last"]],
errors="ignore")

In [30]:
historical_df = pd.json_normalize(historical_data, 
record_path=["terms"],
meta=[["id", "bioguide"],["bio", "birthday"], ["bio","gender"], ["name", "first"], ["name","last"]],
errors="ignore")

In [37]:
current_df["end-type"].value_counts()

special-election    2
Name: end-type, dtype: int64

In [38]:
columns = [
    'type', 
    'start', 
    'end', 
    'state', 
    'district', 
    'party',
    'class',
    'address', 
    'office', 
    'state_rank',
    'id.bioguide', 
    'bio.birthday', 
    'bio.gender', 
    'name.first', 
    'name.last']

In [42]:
df = pd.concat([current_df[columns], historical_df[columns]])

df.columns = [
    'type', 
    'term_start', 
    'term_end', 
    'state', 
    'district', 
    'party',
    'class',
    'address', 
    'office', 
    'state_rank',
    'bioguide_id', 
    'birthdate', 
    'gender', 
    'first_name', 
    'last_name']

In [45]:
df.reset_index(drop=True).to_csv("legislators.csv", index=False)